# Run ngspice simulation exmaples


In [2]:
%%capture out 
importlib.reload(ngspice)

parameters = ('_w8','_w6','_w4','_w10','_w1','_w0',
          '_l8','_l6','_l4','_l10','_l1','_l0',
          "_nf8","_nf6", "_nf4", "_nf10", "_nf1", "_nf0" )
          
values     = ( 1.0000e-06, 7.1800e-05, 1.5700e-05, 2.2000e-06, 1.6000e-06, 9.0000e-06,
           9.4000e-07, 8.8000e-07, 6.7000e-07, 8.9000e-07, 8.9000e-07, 8.4000e-07,
           5.0000e+00, 1.0000e+00, 7.0000e+00, 1.0000e+00, 3.0000e+00, 3.0000e+00)
  
folder = "../circuit_examples/ptm130_ssvcamp_ngspice/"

print(ngspice.simulate(cwd = folder, netlist="open-loop-testbench.cir", param = parameters, val = values))

FileNotFoundError: [Errno 2] No such file or directory: '../circuit_examples/ptm130_ssvcamp_ngspice/cir'

In [1]:
import importlib
import json
import circuits as cir
import optimizers as opt
import ngspice

importlib.reload(cir)
importlib.reload(opt)
importlib.reload(ngspice)




<module 'ngspice' from '/home/nlourenco/Work/workspace-haicas/BAIcO/src/ngspice.py'>

In [ ]:

c = cir.Circuit("../circuit_examples/ptm130_folded_cascode/")
with open("../circuit_examples/ptm130_folded_cascode/sizing_example.json", 'r') as file:
    sizing = json.load(file)

parameters = [ k for  k,v in sizing.items() ]
values     = [ v for  k,v in sizing.items() ]


meas = c.simulate(values, parameters= parameters)


obj,cstr, log = c.target.evaluate(meas)

print(obj, cstr)
for k,v in log['TT'].items():
    print(k, v)





[0.0004646086, -37382850.0, -78.27858] -4.785042125659958
IDD_lt (0.00035, 0.0004646086, -0.32745314285714294)
FOM_gt (500.0, 482.7657085985925, -0.034468582802815034)
VOV_MNM3_gt (0.05, -0.0153057, -1.306114)
VOV_MNM4_gt (0.05, -0.0153055, -1.3061099999999999)
VOV_MNM5_gt (0.05, 0.03674646, -0.2650708)
VOV_MNM6_gt (0.05, 0.03622316, -0.2755368000000001)
VOV_MNM7_gt (0.05, 0.01824278, -0.6351444)
VOV_MNM8_gt (0.05, 0.01824278, -0.6351444)


In [ ]:

nsga2 = opt.NSGA2()

for pop, pop_obj,  pop_cstr, pop_data, evals in nsga2.minimize(c, pop_size=64):
        print(evals)
        print(pop_obj)
        print(pop_cstr)
        print(pop_data[pop_cstr.argmax()][1])



KeyboardInterrupt: 

In [ ]:


import optimizers as opt
import circuits as cir

if __name__ == '__main__':

    nsga2 = opt.NSGA2()

    for pop, pop_obj,  pop_cstr, pop_data, evals, front_no in nsga2.minimize(
        cir.Circuit("./circuit_examples/ptm130_folded_cascode/"), pop_size=32):

        print(evals)
        print(pop_obj[pop_cstr.argmax()] , pop_cstr[pop_cstr.argmax()])
        print(pop_data[pop_cstr.argmax()][1]['TT'])


    quit()


In [ ]:

import importlib
import json
importlib.reload(ngspice)

with open("../circuit_examples/ptm130_folded_cascode/sizing_example.json", 'r') as file:
    sizing = json.load(file)

parameters = [ k for  k,v in sizing.items() ]
values     = [ v for  k,v in sizing.items() ]


folder = "../circuit_examples/ptm130_folded_cascode/"

meas = ngspice.simulate(cwd = folder, netlist=["tb_ac.cir"], param = parameters, val = values)
meas["TT"]["NOISE"] = []





In [ ]:
meas = ngspice.simulate(cwd = folder, netlist="tb_ac2.cir", param = parameters, val = values)
meas["TT"]["NOISE"] = []


for m,v in meas["TT"].items():
    if "VOV" in m or "DELTA" in m: print("{} = {}".format(m, v))

In [ ]:
import numpy as np
import optimizers as opt
import circuits as cir
import ngspice
import importlib
import random

importlib.reload(opt)
importlib.reload(cir)
importlib.reload(ngspice)

seed = 42
np.random.seed(seed)
random.seed(seed)
    
nsga2 = opt.NSGA2()
i = 0
history={}
for pop, pop_obj,  pop_cstr, pop_data, evals, front_no in nsga2.minimize(
        cir.Circuit("../circuit_examples/ptm130_folded_cascode/", corners=None), 
        pop_size=64, evaluations=64*100, mutation=0.1, crossover=0.9):
        
        print(i, pop_cstr[pop_cstr.argmax()])
        history[i] = (pop, pop_obj,  pop_cstr, pop_data, evals, front_no)
        i = i + 1


0 [-3.53570134]
1 [-3.53570134]
2 [-3.42443285]
3 [-3.42443285]
4 [-3.24041173]
5 [-2.37625982]
6 [-2.21312003]
7 [-1.91321226]
8 [-1.20454718]
9 [-1.20454718]
10 [-1.20454718]


In [ ]:
print(history[0][5])



[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [48]:
import numpy


def half_tournemant(rank, cdist):
    n = len(rank)
    index_a = np.arange(n)
    np.random.shuffle(index_a)
    eq_rank = rank[index_a[:n//2]] == rank[index_a[n//2:]]
    larger_cdist = cdist[index_a[:n//2]] > cdist[index_a[n//2:]]

    decision_a = np.logical_or(np.logical_and(eq_rank, larger_cdist),  rank[index_a[:n//2]] < rank[index_a[n//2:]])

    return index_a[np.r_[decision_a, ~decision_a]]
    

rank  = np.array([1, 2, 4, 5, 6, 2, 1, 2])
cdist = np.array([1, 2, 1, 2, 1, 3, 1, 3])
print(rank)
print(cdist)

n = len(rank)
mate = np.zeros(n)
mate[::2] = half_tournemant(rank, cdist)
mate[1::2] = half_tournemant(rank, cdist)


print(mate)

    

##
##    for i in range(n):
##        a = np.random.randint(n)
##        b = np.random.randint(n)
##        if rank[b] == rank[a] and cdist[b] > cdist[a]:
##            a = b
##        if rank[b] < rank[a]:
##            a = b      
##        mate.append(a)
    


[1 2 4 5 6 2 1 2]
[1 2 1 2 1 3 1 3]
[0 1 4 5 6 2 7 3]
[False  True False  True]
[3 0 6 7 2 1 4 5]
[False  True  True False]
[1. 0. 5. 6. 6. 2. 7. 5.]


In [2]:
import os
os.environ['PYTHONPATH'].split(os.pathsep)

['/home/nlourenco/.vscode/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles',
 '/home/nlourenco/.vscode/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles/lib/python']